In [1]:
import bz2
from collections import Counter, namedtuple
import datetime
import numpy
import os
import pandas
from pathlib import Path
import re
from subprocess import run, PIPE
import sys
import zoneinfo

os.environ.setdefault('DJANGO_SETTINGS_MODULE', 'mousedemo.settings')
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"

import django
from django.contrib.auth import get_user_model
from django.db import DEFAULT_DB_ALIAS

MOUSEDEMO = str(Path("mousedemo").absolute())
if MOUSEDEMO not in sys.path:
    sys.path.append(MOUSEDEMO)

django.setup()

from mousedemo import settings
from igvf_mice import models

In [2]:
EC = str(Path("~/proj/encoded_client").expanduser())
if EC not in sys.path:
    sys.path.append(EC)
    
from encoded_client import encoded

In [3]:
server = encoded.ENCODED("api.sandbox.igvf.org")

In [23]:
award = "/awards/HG012077/"
labs = ["/labs/lior-pachter/", "/labs/grant-macgregor/", "/labs/barbara-wold/", "/labs/ali-mortazavi/"]
lab = labs[-1]
jax = "/sources/jackson-labs/"
species = 'Mus musculus'

In [5]:
strain_background = {
    "A/J": "A/J (AJ)",
    "C57BL/6J": "C57BL/6J (B6)",
    "129S1/SvImJ": "129S1/SvImJ (129)",
    "NOD/ShiLtJ": "NOD/ShiLtJ (NOD)",
    "NZO/HlLtJ": "NZO/H1LtJ (NZO)",
    "CAST/EiJ": "CAST/EiJ (CAST)",
    "PWK/PhJ": "PWK/PhJ (PWK)",
    "WSB/EiJ": "WSB/EiJ (WSB)",
    #"CAST (M. m. castaneus)",
    #"WSB (M. m. domesticus)",
    #"PWK (M. m. musculus)",
}

In [6]:
test_run =  "IGVF_003"
plate_003 = models.SplitSeqPlate.objects.get(name=test_run)

In [7]:
tissues = {}
mice = {}
for well in plate_003.splitseqwell_set.all():
    for biosample in well.biosample.all():
        for tissue in biosample.tissue.all():
            tissues[tissue.name] = tissue
            mice[tissue.mouse.name] = tissue.mouse

# rodent_donor

In [32]:
def format_sex(value):
    if models.SexEnum.MALE == value:
        return "male"
    elif models.SexEnum.FEMALE == value:
        return "female"

rodent_donor = []
for mouse_name in sorted(mice):
    mouse = mice[mouse_name]
    dcc_row = {
        #"#response": None,
        #"#response_time": None,
        "accession": None,
        "uuid": None, 
        "aliases:array": f"ali-mortazavi:{mouse.name}",
        "award": award,
        "lab": lab,
        "taxa": species,
        "sex": format_sex(mouse.sex),
        "strain": mouse.strain.name,
        "references": None,
        "url": mouse.strain.url,
        "source": jax,
        "lot_id": None,
        "product_id": mouse.strain.jax_catalog_number,
        "documents": None,
        "alternate_accessions": None,
        "submitter_comment": None,
        "description": None,
        "parents": None,
        "traits": None,
        "phenotypic_features": None,
        "external_resources": None,
        "strain_background": strain_background[mouse.strain.name],
        "genotype": None,
        "individual_rodent": True,
        "rodent_identifier": mouse.name,
    }
    rodent_donor.append(dcc_row)
    
rodent_donor = pandas.DataFrame(rodent_donor)

created = server.post_sheet("rodent_donor", rodent_donor, dry_run=False, verbose=True)
if len(created) > 0:
    rodent_donor.to_excel("rodent_donor.xlsx")


Reponse {'status': 'success', '@type': ['result'], '@graph': [{'aliases': ['ali-mortazavi:016_B6J_10F'], 'award': '/awards/HG012077/', 'lab': '/labs/ali-mortazavi/', 'taxa': 'Mus musculus', 'sex': 'female', 'strain': 'C57BL/6J', 'url': 'http://www.informatics.jax.org/inbred_strains/mouse/docs/C57BL.shtml', 'source': '/sources/jackson-labs/', 'product_id': '000664', 'strain_background': 'C57BL/6J (B6)', 'accession': 'TSTDO36427294', 'status': 'in progress', 'schema_version': '6', 'creation_timestamp': '2023-02-24T00:10:00.158432+00:00', 'submitted_by': '/users/ba3a1b89-ace2-4f6a-95fb-86f55f5cab20/', 'individual_rodent': False, '@id': '/rodent-donors/TSTDO36427294/', '@type': ['RodentDonor', 'Donor', 'Item'], 'uuid': 'a1014f93-da53-46e6-b5a3-f981054e29ec', 'summary': 'TSTDO36427294'}]}
row 0 created: TSTDO36427294
Reponse {'status': 'success', '@type': ['result'], '@graph': [{'aliases': ['ali-mortazavi:017_B6J_10M'], 'award': '/awards/HG012077/', 'lab': '/labs/ali-mortazavi/', 'taxa': 'M

http status: 409
message: b'{"@type": ["HTTPConflict", "Error"], "status": "error", "code": 409, "title": "Conflict", "description": "There was a conflict when trying to complete your request.", "detail": "Keys conflict: [(\'rodentdonor:strain_sex\', \'C57BL/6J/female\')]"}'


HTTPError: 409 Client Error: Conflict for url: https://api.sandbox.igvf.org/rodent_donor

In [59]:
%time r = server.get_json("ali-mortazavi:016_B6J_10F")

CPU times: user 6.51 ms, sys: 4.32 ms, total: 10.8 ms
Wall time: 139 ms


In [58]:
%time r = server._session.head(server.prepare_url("ali-mortazavi:016_B6J_10F"))

CPU times: user 10.2 ms, sys: 377 µs, total: 10.5 ms
Wall time: 101 ms


In [51]:
r.headers

{'Date': 'Sat, 25 Feb 2023 01:10:39 GMT', 'Content-Type': 'text/html; charset=UTF-8', 'Content-Length': '226', 'Connection': 'keep-alive', 'Server': 'nginx/1.21.4', 'Location': 'https://api.sandbox.igvf.org/rodent-donors/TSTDO36427294/', 'X-Request-URL': 'https://api.sandbox.igvf.org/ali-mortazavi:016_B6J_10F', 'X-Stats': 'db_count=1&db_time=2964&es_count=5&es_time=39396&rss_begin=157200384&rss_change=0&rss_end=157200384&wsgi_begin=1677287439868962&wsgi_end=1677287439917325&wsgi_time=48363'}

In [35]:
created = server.post_sheet("rodent_donor", rodent_donor, dry_run=False, verbose=True)
if len(created) > 0:
    rodent_donor.to_excel("rodent_donor.xlsx")

http status: 409
message: b'{"@type": ["HTTPConflict", "Error"], "status": "error", "code": 409, "title": "Conflict", "description": "There was a conflict when trying to complete your request.", "detail": "Keys conflict: [(\'rodentdonor:strain_sex\', \'C57BL/6J/female\')]"}'


HTTPError: 409 Client Error: Conflict for url: https://api.sandbox.igvf.org/rodent_donor

In [36]:
%debug

> /usr/lib/python3/dist-packages/requests/models.py(943)raise_for_status()
    941 
    942         if http_error_msg:
--> 943             raise HTTPError(http_error_msg, response=self)
    944 
    945     def close(self):

ipdb> up
> /woldlab/loxcyc/home/diane/proj/encoded_client/encoded_client/encoded.py(461)post_json()
    459             LOGGER.error("http status: {}".format(response.status_code))
    460             LOGGER.error("message: {}".format(response.content))
--> 461             response.raise_for_status()
    462         return response.json()
    463 

ipdb> p payload
'{"aliases": ["ali-mortazavi:018_B6J_10F"], "award": "/awards/HG012077/", "lab": "/labs/ali-mortazavi/", "taxa": "Mus musculus", "sex": "female", "strain": "C57BL/6J", "url": "http://www.informatics.jax.org/inbred_strains/mouse/docs/C57BL.shtml", "source": "/sources/jackson-labs/", "product_id": "000664", "strain_background": "C57BL/6J (B6)"}'
ipdb> c


[]

# suggest sample terms

In [11]:
sample_terms = []
for term in models.OntologyTerm.objects.all():
    dcc_row = {
        "term_id": term.curie,
        "term_name": term.name,
        "description": term.description,
    }
    sample_terms.append(dcc_row)

sample_terms = pandas.DataFrame(sample_terms)
sample_terms.to_excel("sample_terms.xlsx", index=False)
sample_terms

,term_id,term_name,description
0,UBERON:0001898,hypothalamus,A specialized brain region of the ventral dien...
1,UBERON:0000007,pituitary gland,The pituitary gland is an endocrine gland that...
2,UBERON:0002037,cerebellum,Part of the metencephalon that lies in the pos...
3,NTR:0000646,left cerebral cortex,None
4,NTR:0000750,Hippocampal formation left,None
5,NTR:0000647,right cerebral cortex,None
6,NTR:0000751,Hippocampal formation right,None
7,UBERON:0002107,liver,An exocrine gland which secretes bile and func...
8,UBERON:0000948,heart,A myogenic muscular circulatory organ found in...
9,UBERON:0002048,lung,Respiration organ that develops as an outpocke...


# tissue

In [40]:
tissue_sheet = []
for tissue_name in tissues:
    tissue = tissues[tissue_name]
    donor_alias = f"ali-mortazavi:{tissue.mouse.name}"
    for term in tissue.ontology_term.all():
        curie = term.curie.replace(":", "_")
        alias = f"ali-mortazavi:{tissue.name}_{curie}"
        sample_term = "/sample-terms/{}/".format(curie)
        dcc_row = {
            "accession": None,
            "uuid": None,
            "aliases:array": alias,
            "award": award,
            "lab": lab,
            "source": jax,
            "donors:array": donor_alias,
            "taxa": species,
            "biosample_term": sample_term,
            "term_names:skip": term.name,
        }
        tissue_sheet.append(dcc_row)
    
tissue_sheet = pandas.DataFrame(tissue_sheet)

#server.post_sheet("tissue", tissue_sheet, dry_run=True, verbose=True)
tissue_sheet

,accession,uuid,aliases:array,award,lab,source,donors:array,taxa,biosample_term,term_names:skip
0,None,None,ali-mortazavi:016_B6J_10F_03_NTR_0000646,/awards/HG012077/,/labs/ali-mortazavi/,/sources/jackson-labs/,ali-mortazavi:016_B6J_10F,Mus musculus,/sample-terms/NTR_0000646/,left cerebral cortex
1,None,None,ali-mortazavi:016_B6J_10F_03_NTR_0000750,/awards/HG012077/,/labs/ali-mortazavi/,/sources/jackson-labs/,ali-mortazavi:016_B6J_10F,Mus musculus,/sample-terms/NTR_0000750/,Hippocampal formation left
2,None,None,ali-mortazavi:017_B6J_10M_03_NTR_0000646,/awards/HG012077/,/labs/ali-mortazavi/,/sources/jackson-labs/,ali-mortazavi:017_B6J_10M,Mus musculus,/sample-terms/NTR_0000646/,left cerebral cortex
3,None,None,ali-mortazavi:017_B6J_10M_03_NTR_0000750,/awards/HG012077/,/labs/ali-mortazavi/,/sources/jackson-labs/,ali-mortazavi:017_B6J_10M,Mus musculus,/sample-terms/NTR_0000750/,Hippocampal formation left
4,None,None,ali-mortazavi:018_B6J_10F_03_NTR_0000646,/awards/HG012077/,/labs/ali-mortazavi/,/sources/jackson-labs/,ali-mortazavi:018_B6J_10F,Mus musculus,/sample-terms/NTR_0000646/,left cerebral cortex
...,...,...,...,...,...,...,...,...,...,...
187,None,None,ali-mortazavi:051_NZOJ_10M_06_UBERON_0000948,/awards/HG012077/,/labs/ali-mortazavi/,/sources/jackson-labs/,ali-mortazavi:051_NZOJ_10M,Mus musculus,/sample-terms/UBERON_0000948/,heart
188,None,None,ali-mortazavi:096_WSBJ_10F_06_UBERON_0000948,/awards/HG012077/,/labs/ali-mortazavi/,/sources/jackson-labs/,ali-mortazavi:096_WSBJ_10F,Mus musculus,/sample-terms/UBERON_0000948/,heart
189,None,None,ali-mortazavi:052_NZOJ_10F_06_UBERON_0000948,/awards/HG012077/,/labs/ali-mortazavi/,/sources/jackson-labs/,ali-mortazavi:052_NZOJ_10F,Mus musculus,/sample-terms/UBERON_0000948/,heart
190,None,None,ali-mortazavi:063_WSBJ_10M_06_UBERON_0000948,/awards/HG012077/,/labs/ali-mortazavi/,/sources/jackson-labs/,ali-mortazavi:063_WSBJ_10M,Mus musculus,/sample-terms/UBERON_0000948/,heart


In [41]:
server.post_sheet("tissue", tissue_sheet.iloc[0:4], dry_run=False, verbose=True)
tissue_sheet.iloc[0:4]

Reponse {'status': 'success', '@type': ['result'], '@graph': [{'aliases': ['ali-mortazavi:016_B6J_10F_03_NTR_0000646'], 'award': '/awards/HG012077/', 'lab': '/labs/ali-mortazavi/', 'source': '/sources/jackson-labs/', 'donors': ['/rodent-donors/TSTDO36427294/'], 'taxa': 'Mus musculus', 'biosample_term': '/sample-terms/NTR_0000646/', 'accession': 'TSTSM69861046', 'status': 'in progress', 'schema_version': '9', 'creation_timestamp': '2023-02-24T00:21:07.187234+00:00', 'submitted_by': '/users/ba3a1b89-ace2-4f6a-95fb-86f55f5cab20/', '@id': '/tissues/TSTSM69861046/', '@type': ['Tissue', 'Biosample', 'Sample', 'Item'], 'uuid': '2e8be4be-3595-4b00-a267-e2a5a195e620', 'summary': 'TSTSM69861046', 'sex': 'female', 'age': 'unknown'}]}
row 0 created: TSTSM69861046


/usr/lib/python3/dist-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)
/woldlab/loxcyc/home/diane/proj/encoded_client/encoded_client/encoded.py:508: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sheet.loc[i, "accession"] = accession
/woldlab/loxcyc/home/diane/proj/encoded_client/encoded_client/encoded.py:510: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  shee

Reponse {'status': 'success', '@type': ['result'], '@graph': [{'aliases': ['ali-mortazavi:016_B6J_10F_03_NTR_0000750'], 'award': '/awards/HG012077/', 'lab': '/labs/ali-mortazavi/', 'source': '/sources/jackson-labs/', 'donors': ['/rodent-donors/TSTDO36427294/'], 'taxa': 'Mus musculus', 'biosample_term': '/sample-terms/NTR_0000750/', 'accession': 'TSTSM75831182', 'status': 'in progress', 'schema_version': '9', 'creation_timestamp': '2023-02-24T00:21:07.457425+00:00', 'submitted_by': '/users/ba3a1b89-ace2-4f6a-95fb-86f55f5cab20/', '@id': '/tissues/TSTSM75831182/', '@type': ['Tissue', 'Biosample', 'Sample', 'Item'], 'uuid': '6ff911bd-3f2b-4c5f-8ff4-58ce870de1c4', 'summary': 'TSTSM75831182', 'sex': 'female', 'age': 'unknown'}]}
row 1 created: TSTSM75831182
Reponse {'status': 'success', '@type': ['result'], '@graph': [{'aliases': ['ali-mortazavi:017_B6J_10M_03_NTR_0000646'], 'award': '/awards/HG012077/', 'lab': '/labs/ali-mortazavi/', 'source': '/sources/jackson-labs/', 'donors': ['/rodent-d

,accession,uuid,aliases:array,award,lab,source,donors:array,taxa,biosample_term,term_names:skip
0,TSTSM69861046,2e8be4be-3595-4b00-a267-e2a5a195e620,ali-mortazavi:016_B6J_10F_03_NTR_0000646,/awards/HG012077/,/labs/ali-mortazavi/,/sources/jackson-labs/,ali-mortazavi:016_B6J_10F,Mus musculus,/sample-terms/NTR_0000646/,left cerebral cortex
1,TSTSM75831182,6ff911bd-3f2b-4c5f-8ff4-58ce870de1c4,ali-mortazavi:016_B6J_10F_03_NTR_0000750,/awards/HG012077/,/labs/ali-mortazavi/,/sources/jackson-labs/,ali-mortazavi:016_B6J_10F,Mus musculus,/sample-terms/NTR_0000750/,Hippocampal formation left
2,TSTSM69542835,d20b65ea-b17c-46af-8f80-86337efbe4c1,ali-mortazavi:017_B6J_10M_03_NTR_0000646,/awards/HG012077/,/labs/ali-mortazavi/,/sources/jackson-labs/,ali-mortazavi:017_B6J_10M,Mus musculus,/sample-terms/NTR_0000646/,left cerebral cortex
3,TSTSM28050193,2b8d64b6-0e1e-4ef3-a733-c92c8b5799b8,ali-mortazavi:017_B6J_10M_03_NTR_0000750,/awards/HG012077/,/labs/ali-mortazavi/,/sources/jackson-labs/,ali-mortazavi:017_B6J_10M,Mus musculus,/sample-terms/NTR_0000750/,Hippocampal formation left


In [45]:
tissue_sheet[tissue_sheet["donors:array"] == "ali-mortazavi:016_B6J_10F"]

,accession,uuid,aliases:array,award,lab,source,donors:array,taxa,biosample_term,term_names:skip
0,TSTSM69861046,2e8be4be-3595-4b00-a267-e2a5a195e620,ali-mortazavi:016_B6J_10F_03_NTR_0000646,/awards/HG012077/,/labs/ali-mortazavi/,/sources/jackson-labs/,ali-mortazavi:016_B6J_10F,Mus musculus,/sample-terms/NTR_0000646/,left cerebral cortex
1,TSTSM75831182,6ff911bd-3f2b-4c5f-8ff4-58ce870de1c4,ali-mortazavi:016_B6J_10F_03_NTR_0000750,/awards/HG012077/,/labs/ali-mortazavi/,/sources/jackson-labs/,ali-mortazavi:016_B6J_10F,Mus musculus,/sample-terms/NTR_0000750/,Hippocampal formation left
128,None,None,ali-mortazavi:016_B6J_10F_06_UBERON_0000948,/awards/HG012077/,/labs/ali-mortazavi/,/sources/jackson-labs/,ali-mortazavi:016_B6J_10F,Mus musculus,/sample-terms/UBERON_0000948/,heart


In [42]:
tissue_sheet.to_excel("tissue_sheet.xlsx")

# measurement_set

In [13]:
platform_decoder = {
    "NE": "nextseq",
    "NO": "novaseq",
    "ON": "nanopore",
    "PB": "pacbio",
}

measurement_plates = {}
for run in models.SequencingRun.objects.all():
    platform = platform_decoder[run.platform]
    alias = f"ali-mortazavi:{platform}_{run.name}"
    measurement_plates.setdefault(alias, set()).add(run.library.plate)
    
    
for alias in measurement_plates:
    assert len(measurement_plates[alias]) == 1

print(measurement_plates)

{'ali-mortazavi:nextseq_1049': {<SplitSeqPlate: SplitSeqPlate object (1)>}, 'ali-mortazavi:novaseq_2022-12-16': {<SplitSeqPlate: SplitSeqPlate object (1)>}, 'ali-mortazavi:nextseq_1051': {<SplitSeqPlate: SplitSeqPlate object (2)>}, 'ali-mortazavi:novaseq_2022-12-21': {<SplitSeqPlate: SplitSeqPlate object (2)>}, 'ali-mortazavi:nextseq_1053': {<SplitSeqPlate: SplitSeqPlate object (3)>}, 'ali-mortazavi:novaseq_2023-02-03': {<SplitSeqPlate: SplitSeqPlate object (3)>}, 'ali-mortazavi:nextseq_1054': {<SplitSeqPlate: SplitSeqPlate object (4)>}}


In [14]:
plate = next(iter(measurement_plates[alias]))

In [15]:
def get_samples_donors(plate):
    samples = []
    donors = []
    for well in plate.splitseqwell_set.all():
        for biosample in well.biosample.all():
            for tissue in biosample.tissue.all():
                for term in tissue.ontology_term.all():
                    curie = term.curie.replace(":", "_")
                    tissue_alias = f"ali-mortazavi:{tissue.name}_{curie}"
                    samples.append(tissue_alias)
                    mouse_alias = f"ali-mortazavi:{tissue.mouse.name}"
                    donors.append(mouse_alias)
    return {
        "samples": samples,
        "donor": donors
    }

In [16]:
measurements = []

for alias in measurement_plates:
    plate = next(iter(measurement_plates[alias]))
    if plate.name == test_run:
        plate_details = get_samples_donors(plate)
        
        dcc_row = {
            "accession": None,
            "uuid": None,
            "aliases": alias,
            "award": award,
            "lab": lab,
            "assay_term": "/assay-terms/OBI_0003109/", # single-nucleus RNA sequencing assay
            "documents": None,
            "alternate_accessions": None,
            "submitter_comment": None,
            "description": None,
            "samples:array": plate_details["samples"],
            "donors:array": plate_details["donor"],
            "protocol": None,
        }
        measurements.append(dcc_row)
        
measurements = pandas.DataFrame(measurements)
server.post_sheet("measurement_set", tissue_sheet, dry_run=True, verbose=True)
measurements

Validation error row 0


ValidationError: 'assay_term' is a required property

Failed validating 'required' in schema:
    {'$id': '/profiles/measurement_set.json',
     '$schema': 'https://json-schema.org/draft/2020-12/schema',
     '@type': ['JSONSchema'],
     'additionalProperties': False,
     'boost_values': {'@type': 1.0, 'assay_term': 1.0, 'protocol': 1.0},
     'changelog': '/profiles/changelogs/measurement_set.md',
     'dependentSchemas': {'donors': {'comment': 'Specification of donors '
                                                'is mutually exclusive '
                                                'with specification of '
                                                'samples.',
                                     'not': {'required': ['samples']}},
                          'samples': {'comment': 'Specification of samples '
                                                 'is mutually exclusive '
                                                 'with specification of '
                  

In [46]:
measurements.to_excel("measurement_set.xlsx", index=False)